## Open-Clusters Exploration

In this notebook we explore the open-clusters available within APOGEE dr16 and we generate a pickled dataset for working with them.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import inspect

from apoNN.src.occam import Occam

import apogee.tools.read as apread
import apogee.tools.path as apogee_path
from apogee.tools import bitmask
import collections


from apoNN.src.datasets import ApogeeDataset
from apoNN.src.utils import generate_loss_with_masking
from apoNN.src.utils import dump
import apoNN.src.vectors as vector

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

apogee_path.change_dr(16)

## Generating filtered dataset

The first step of the process is to generate a reduced allStar containing only those stars in open clusters

In [ ]:
def prepare_occam_allStar(occam_kept,allStar):
    """
    Given a list of of occam_ids, generates a filtered APOGEE allStar containing only those OCCAM stars.
    Also filters out stars within allStar without spectra. 
    
    INPUTS
    ------
    occam: Occam object
    occam_kept: array
        boolean mask of same size of occam dataset with True for used stars and False for discarded Stars
    allStar:
        allStar file
        
    OUTPUTS
    -------
    (1) filtered_allStar containing only open cluster stars
    (2) cluster_ids of every star in the filtered dataset
    """
    occam = Occam()
    filtered_occam_apogee_id = np.array(occam.apogee_id)[occam_kept]
    list_apogee_ids = list(allStar["Apogee_id"])
    apogee_idxs = []
    for idx in list(filtered_occam_apogee_id):
        try:
            apogee_idx = list_apogee_ids.index(idx)
            apogee_id,loc,telescope = allStar[apogee_idx]["APOGEE_ID"],allStar[apogee_idx]["FIELD"], allStar[apogee_idx]["TELESCOPE"]
            apread.aspcapStar(loc_id=str(loc),apogee_id=apogee_id,telescope=telescope,ext=1)
            apogee_idxs.append(apogee_idx)
        except:
            apogee_idxs.append(-1)
    apogee_idxs = np.array(apogee_idxs)
    found_mask = apogee_idxs!=-1
    
    return allStar[apogee_idxs[found_mask]],np.array(occam.cluster_id)[occam_kept][found_mask]


In [ ]:
occam = Occam()
allStar= apread.allStar()

In [ ]:
occam_kept = occam.cg_prob>0.5
#occam_kept = (occam.feh_prob<0.6)  & (occam.feh_prob>0.58)
print(sum(occam_kept))

In [ ]:
upper_temp_cut = allStar["Teff"]<5000
lower_temp_cut = allStar["Teff"]>4000
lower_g_cut = allStar["logg"]>1.5
upper_g_cut = allStar["logg"]<3

combined_cut = lower_g_cut & upper_g_cut & lower_temp_cut & upper_temp_cut 


In [ ]:
occam_allStar,occam_cluster_idxs = prepare_occam_allStar(occam_kept,allStar[combined_cut])

In [ ]:
occam_allStar

In [ ]:
len(occam_allStar)

In [ ]:
def make_registry(cluster_names):
    clusters = list(set(cluster_names))
    cluster_registry = {}
    for cluster in clusters:
        cluster_idxs = np.where(cluster_names==cluster)
        cluster_registry[cluster] = cluster_idxs[0]
    return cluster_registry



In [ ]:
registry = make_registry(occam_cluster_idxs)

In [ ]:
one_star_cluster_ids = []
for key in registry:
    if len(registry[key])==1:
        one_star_cluster_ids.append(registry[key][0])
    print(registry[key])

In [ ]:
multiple_star_cluster_ids = np.delete(np.arange(len(occam_allStar)),one_star_cluster_ids)

In [ ]:
occam_allStar[multiple_star_cluster_ids]

In [ ]:
filtered_occam_cluster_idxs  = occam_cluster_idxs[multiple_star_cluster_ids]

In [ ]:
make_registry(filtered_occam_cluster_idxs)

In [ ]:
one_star_cluster_ids

In [ ]:
occam = {"allStar":occam_allStar,
        "cluster_idxs":occam_cluster_idxs}

In [ ]:
dump(occam,"occam")

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

import apogee.tools.read as apread
import apogee.tools.path as apogee_path
from apogee.tools import bitmask
import collections


from apoNN.src.datasets import ApogeeDataset
from apoNN.src.utils import generate_loss_with_masking,dump
import apoNN.src.vectors as vector
import apoNN.src.occam as occam_utils


import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

apogee_path.change_dr(16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
occam = occam_utils.Occam()
allStar= apread.allStar()

/share/splinter/ddm/modules/apogee/apogee/tools/read.py:303: RuntimeWarning: Extinction-corrected J,H,K not added because esutil is not installed
  warnings.warn("Extinction-corrected J,H,K not added because esutil is not installed",RuntimeWarning)


In [10]:
occam_kept = occam.cg_prob>0.8
#occam_kept = (occam.feh_prob<0.6)  & (occam.feh_prob>0.58)
print(sum(occam_kept))

1444


In [5]:
upper_temp_cut = allStar["Teff"]<5000
lower_temp_cut = allStar["Teff"]>4000
lower_g_cut = allStar["logg"]>1.5
upper_g_cut = allStar["logg"]<3

combined_cut = lower_g_cut & upper_g_cut & lower_temp_cut & upper_temp_cut 


In [6]:
occam_allStar,occam_cluster_idxs = occam_utils.prepare_occam_allStar(occam_kept,allStar[combined_cut])

In [13]:
registry = vector.OccamLatentVector.make_registry(occam_cluster_idxs)

In [14]:
one_star_cluster_ids = []
for key in registry:
    if len(registry[key])==1:
        one_star_cluster_ids.append(registry[key][0])

[56 57 58 59 60 61 62 63]
[84 85 86 87 88 89 90 91 92 93]
[11 12]
[76 77 78 79 80]
[19 20 21 22]
[102 103]
[193]
[30 31 32]
[ 94  95  96  97  98  99 100 101]
[83]
[146]
[176]
[23]
[81 82]
[18]
[36 37 38 39 40 41]
[13]
[0]
[25 26]
[177 178 179 180 181 182 183 184 185 186 187 188 189 190]
[24]
[15 16 17]
[35]
[194 195 196 197 198 199 200]
[64 65 66 67 68 69 70 71 72 73 74 75]
[28 29]
[106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 144 145]
[14]
[27]
[191]
[192]
[10]
[147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164
 165 166 167 168 169 170 171 172 173 174 175]
[33 34]
[42 43 44 45 46 47 48 49 50 51 52 53 54 55]
[104 105]
[1 2 3 4 5 6 7 8 9]


In [15]:
one_star_cluster_ids

[193, 83, 146, 176, 23, 18, 13, 0, 24, 35, 14, 27, 191, 192, 10]

In [16]:
multiple_star_cluster_ids = np.delete(np.arange(len(occam_allStar)),one_star_cluster_ids)

In [21]:
filtered_occam_allStar=  occam_allStar[multiple_star_cluster_ids]
filtered_occam_cluster_idxs  = occam_cluster_idxs[multiple_star_cluster_ids]